In [1]:
import logging
import matplotlib
import multiprocessing as mp
import sys
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.backends.cudnn
import torch.utils.data
import torchvision.transforms

import utils.binvox_visualization
import utils.data_loaders
import utils.data_transforms
import utils.network_utils

# Fix problem: no $DISPLAY environment variable
matplotlib.use('Agg')

from argparse import ArgumentParser
from datetime import datetime as dt
from pprint import pprint

from config import cfg
from core.train import train_net
#from core.test import test_net
#from core.inference import inference_net
from core.demo import test_net
#from core.demo_trial import test_net
from torch.utils.tensorboard import SummaryWriter
import skimage.measure as sm

/home/sidroy/anaconda3/envs/3d_vision/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sidroy/anaconda3/envs/3d_vision/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sidroy/anaconda3/envs/3d_vision/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sidroy/ana

In [2]:
PATH = 'pretrained_models/ckpt-epoch-0550.pth'
torch.backends.cudnn.benchmark = True

#checkpoint = (torch.load(PATH))

print('Use config:')
pprint(cfg)

cfg.CONST.WEIGHTS = './pretrained_models/Pix2Vox-A-ShapeNet_old.pth'
#cfg.CONST.WEIGHTS = './pretrained_models/ckpt-epoch-0550.pth'

writer = SummaryWriter('./output/tensorboard')
generated_volume, rendering_images = test_net(cfg,output_dir='./output')

print(generated_volume.shape)
volume = generated_volume.reshape(32,32,32)
print(volume.shape)

Use config:
{'CONST': {'BATCH_SIZE': 64,
           'CROP_IMG_H': 128,
           'CROP_IMG_W': 128,
           'DEVICE': '0',
           'IMG_H': 224,
           'IMG_W': 224,
           'N_VIEWS_RENDERING': 2,
           'N_VOX': 32,
           'RNG_SEED': 0},
 'DATASET': {'MEAN': [0.5, 0.5, 0.5],
             'STD': [0.5, 0.5, 0.5],
             'TEST_DATASET': 'ShapeNet'},
 'DATASETS': {'INFERENCE': {'RENDERING_PATH': './datasets/DemoImage/%s/%s/rendering/%02d.png',
                            'TAXONOMY_FILE_PATH': './datasets/DemoData.json'},
              'PASCAL3D': {'ANNOTATION_PATH': '/home/sidroy/software/Pix2Vox/datasets/PASCAL3D/Annotations/%s_imagenet/%s.mat',
                           'RENDERING_PATH': '/home/sidroy/software/Pix2Vox/datasets/PASCAL3D/Images/%s_imagenet/%s.JPEG',
                           'TAXONOMY_FILE_PATH': './datasets/Pascal3D.json',
                           'VOXEL_PATH': '/home/sidroy/software/Pix2Vox/datasets/PASCAL3D/CAD/%s/%02d.binvox'},
      

ValueError: too many values to unpack (expected 3)

In [ ]:
img_array = rendering_images.cpu().numpy()
print(img_array.shape)
for i in range(4,5):
    image_array = img_array[:,i,:,:,:]
    print(image_array.shape)
    image_array = image_array.reshape(image_array.shape[2],image_array.shape[3],3)
    print(image_array.shape)
    %matplotlib notebook
    fig = plt.figure(i)
    plt.imshow(image_array)
    print("mean = {}".format(np.mean(image_array)))

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.gca(projection='3d')
#ax.set_aspect('equal')
volume_plot = volume.squeeze().__ge__(0.1)
ax.voxels(volume_plot, facecolor='k', edgecolor="b")
ax.view_init(0,70)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.axis('off')
plt.show()
plt.savefig("3D_reco.png", bbox_inches='tight')

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.gca(projection='3d')
#ax.set_aspect('equal')
volume_plot = volume.squeeze().__ge__(0.3)
ax.voxels(volume_plot, facecolor='k', edgecolor="b")
ax.view_init(0,70)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.axis('off')
plt.show()
plt.savefig("3D_reco.png", bbox_inches='tight')


In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.gca(projection='3d')
#ax.set_aspect('equal')
volume_plot = volume.squeeze().__ge__(0.35)
ax.voxels(volume_plot, facecolor='k', edgecolor="b")
ax.view_init(0,70)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.axis('off')
plt.show()
plt.savefig("3D_reco.png", bbox_inches='tight')

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.gca(projection='3d')
#ax.set_aspect('equal')
volume = volume.squeeze().__ge__(0.3)
ax.voxels(volume, facecolor='r',edgecolor="k")
ax.view_init(30, 240)
plt.show()

In [ ]:
verts, faces = createMesh(volume)

%matplotlib notebook
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(verts[:, 0], verts[:,1], faces, verts[:, 2], linewidth=0.2, antialiased=True)

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.gca(projection='3d')
#ax.set_aspect('equal')
volume = volume.squeeze().__ge__(0.3)
ax.voxels(volume, facecolor='r',edgecolor="k")
ax.view_init(30, 240)
plt.show()

In [ ]:
verts, faces = createMesh(volume.squeeze().__ge__(0.2))

%matplotlib notebook
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(verts[:, 0], verts[:,1], faces, verts[:, 2], linewidth=0.2, antialiased=True)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.axis('off')

In [ ]:
def createMesh(vox, step=1, threshold = 0.5) :   

    vox = np.pad(vox, step)

    verts, faces, normals, values = sm.marching_cubes_lewiner(vox, 0.5, step_size=step)

    return verts, faces

In [ ]:
verts, faces = createMesh(volume)

%matplotlib notebook
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(verts[:, 0], verts[:,1], faces, verts[:, 2], linewidth=0.2, antialiased=True)

In [ ]:
plt.imshow(rendering_images)